In [1]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
import sklearn as skl

from sklearn.model_selection import cross_val_score

In [24]:
df = pd.read_csv("../datasets/train_radiomics_hipocamp.csv")

In [3]:
df.columns
df.dtypes

ID                                   object
Image                                object
Mask                                 object
diagnostics_Versions_PyRadiomics     object
diagnostics_Versions_Numpy           object
                                     ...   
lbp-3D-k_ngtdm_Contrast             float64
lbp-3D-k_ngtdm_Strength             float64
Sex                                   int64
Age                                 float64
Transition                           object
Length: 2181, dtype: object

In [4]:
df['Transition']

0        CN-CN
1        CN-CN
2        AD-AD
3       CN-MCI
4        CN-CN
        ...   
300      CN-CN
301      CN-CN
302     MCI-AD
303    MCI-MCI
304      CN-CN
Name: Transition, Length: 305, dtype: object

In [36]:
# Selecionar 10 amostras aleatórias de cada categoria
n_samples = 10
df = df.groupby('Transition').sample(n=n_samples, random_state=2021)
df['Transition'].value_counts()

Transition
AD-AD      10
CN-CN      10
CN-MCI     10
MCI-AD     10
MCI-MCI    10
Name: count, dtype: int64

In [26]:
X = df.drop(['Transition'], axis=1)     
y = df['Transition'].to_frame()         

In [27]:
df_numerico = X.select_dtypes(include=[np.number])

from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = MinMaxScaler()
df_numerico = pd.DataFrame(scaler.fit_transform(df_numerico), columns=df_numerico.columns, index=df_numerico.index) #normalização
df_numerico.head()

,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,...,lbp-3D-k_glszm_ZoneEntropy,lbp-3D-k_glszm_ZonePercentage,lbp-3D-k_glszm_ZoneVariance,lbp-3D-k_ngtdm_Busyness,lbp-3D-k_ngtdm_Coarseness,lbp-3D-k_ngtdm_Complexity,lbp-3D-k_ngtdm_Contrast,lbp-3D-k_ngtdm_Strength,Sex,Age
110,0.0,0.246449,0.0,0.459770,0.244510,0.0,0.171744,0.111061,0.248900,0.533767,...,0.320419,0.320741,0.294135,0.310111,0.433441,0.547022,0.488658,0.433929,0.0,0.253731
251,0.0,0.570939,0.0,0.241379,0.172289,0.0,0.499864,0.356342,0.369524,0.349516,...,0.424655,0.563725,0.107549,0.240954,0.515151,0.511401,0.425503,0.510575,1.0,0.656716
282,0.0,0.712351,0.0,0.000000,0.476341,0.0,0.000000,0.423143,0.568471,0.496146,...,0.306096,0.269993,0.492617,0.567850,0.203783,0.717575,0.674283,0.202072,0.0,0.000000
81,0.0,0.583094,0.0,0.896552,0.631198,0.0,0.662583,0.540824,0.832354,0.642508,...,0.695809,0.487897,0.455848,0.423261,0.263740,0.469722,0.373690,0.267255,1.0,0.164179
129,0.0,0.402368,0.0,0.701149,0.144442,0.0,0.445741,0.579573,0.559746,0.296001,...,0.356537,1.000000,0.000000,0.033007,0.925360,0.156428,0.110364,0.914894,1.0,1.000000


In [28]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Treinar um modelo Random Forest
model = RandomForestClassifier(random_state=2021)
model.fit(df_numerico, y)

# Obter as importâncias das features
feature_importances = pd.DataFrame({
    'Feature': df_numerico.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Mostrar as 10 features mais importantes
#print(feature_importances.head(10))

# Selecionar as n melhores features
n = 500
features_to_keep = feature_importances.iloc[:n]['Feature']

# Filtrar o DataFrame original para manter somente essas features
df_numerico = df_numerico[features_to_keep]

df_numerico.describe()



c:\Users\super\anaconda3\envs\myenv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,exponential_glrlm_ShortRunEmphasis,log-sigma-2-0-mm-3D_firstorder_InterquartileRange,log-sigma-2-0-mm-3D_gldm_SmallDependenceEmphasis,log-sigma-1-0-mm-3D_gldm_SmallDependenceEmphasis,log-sigma-2-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation,wavelet-LLL_glrlm_RunVariance,exponential_glrlm_ShortRunHighGrayLevelEmphasis,log-sigma-5-0-mm-3D_glszm_GrayLevelNonUniformity,gradient_gldm_LargeDependenceLowGrayLevelEmphasis,wavelet-LHL_glszm_SmallAreaLowGrayLevelEmphasis,...,log-sigma-1-0-mm-3D_glcm_MCC,wavelet-HLH_glrlm_LongRunLowGrayLevelEmphasis,lbp-3D-k_gldm_DependenceEntropy,wavelet-LLL_glcm_ClusterShade,gradient_glszm_ZoneVariance,gradient_gldm_SmallDependenceEmphasis,gradient_glrlm_LongRunLowGrayLevelEmphasis,squareroot_glszm_HighGrayLevelZoneEmphasis,log-sigma-1-0-mm-3D_glcm_ClusterProminence,wavelet-LLL_glcm_Imc1
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.365246,0.352586,0.513483,0.459801,0.336730,0.449920,0.154557,0.297561,0.588549,0.212812,...,0.524983,0.859991,0.578118,0.804919,0.351732,0.449213,0.438192,0.442924,0.300954,0.682699
std,0.208117,0.250004,0.230593,0.243728,0.236620,0.243788,0.156312,0.239582,0.259883,0.220703,...,0.236632,0.182638,0.253223,0.177237,0.245371,0.243148,0.251573,0.259060,0.227161,0.223432
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.204563,0.182546,0.376721,0.302733,0.171235,0.293439,0.076470,0.125409,0.364367,0.092725,...,0.340028,0.849302,0.408586,0.718427,0.165817,0.250619,0.206104,0.219437,0.117435,0.595148
50%,0.373305,0.290418,0.474745,0.443112,0.263307,0.409289,0.118607,0.215361,0.651031,0.155672,...,0.552462,0.882324,0.583042,0.856271,0.332098,0.452115,0.464077,0.480050,0.264791,0.724285
75%,0.491711,0.512380,0.647886,0.604118,0.513195,0.592260,0.182158,0.396413,0.757176,0.206316,...,0.659338,0.933209,0.770622,0.932154,0.507666,0.588829,0.598224,0.578412,0.394629,0.834844
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
#Retirar uma das features de um par com mais de 80% de correlação
corr_matrix = df_numerico.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
df_numerico = df_numerico.drop(columns=to_drop)

In [40]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np

# Definindo o classificador
clf = DecisionTreeClassifier(criterion='gini', max_depth=10, random_state=2021)

# Listas para armazenar as métricas
scores = []
f1_scores = []

# Utilizando StratifiedKFold para garantir o balanceamento entre as classes
kf = StratifiedKFold(n_splits=3)

# Loop de treinamento e avaliação
for train, test in kf.split(df_numerico, y):
    clf.fit(df_numerico.iloc[train], y.iloc[train])  # Ajustando as divisões corretamente
    score = clf.score(df_numerico.iloc[train], y.iloc[train])
    scores.append(score)
    
    y_predicted = clf.predict(df_numerico.iloc[test])  # Fazendo a predição
    print("Confusion Matrix:")
    print(confusion_matrix(y.iloc[test], y_predicted))  # Confusion Matrix para o conjunto de teste
    
    f1 = f1_score(y.iloc[test], y_predicted, average='weighted')  # F1-Score ponderado para desbalanceamento
    f1_scores.append(f1)
    print(f"F1-Score: {f1:.2f}")
    print(f"Train Accuracy: {score:.2f}")

# Resultados finais
print("\nRESULT: %0.2f accuracy with a std deviation of %0.2f" % (np.mean(scores), np.std(scores)))
print("F1-Score: %0.2f with a std deviation of %0.2f" % (np.mean(f1_scores), np.std(f1_scores)))


Confusion Matrix:
[[4 0 0 0 0]
 [0 1 0 0 2]
 [0 0 1 0 2]
 [1 0 0 1 2]
 [0 0 0 1 2]]
F1-Score: 0.53
Train Accuracy: 1.00
Confusion Matrix:
[[1 1 0 1 0]
 [0 3 1 0 0]
 [0 2 1 0 0]
 [0 0 1 2 0]
 [0 1 0 2 1]]
F1-Score: 0.46
Train Accuracy: 1.00
Confusion Matrix:
[[3 0 0 0 0]
 [1 1 0 0 1]
 [0 0 3 1 0]
 [0 0 1 1 1]
 [1 0 0 1 1]]
F1-Score: 0.55
Train Accuracy: 1.00

RESULT: 1.00 accuracy with a std deviation of 0.00
F1-Score: 0.51 with a std deviation of 0.04


In [7]:

clf = DecisionTreeClassifier(random_state=2021)
clf.fit(df_numerico, y)
print(clf.get_depth())

10


In [8]:
X_test = pd.read_csv("../datasets/test_radiomics_hipocamp.csv")
X_test = X_test.select_dtypes(include=[np.number])

In [9]:
predictions = clf.predict(X_test)

In [10]:
predictions

array(['MCI-AD', 'MCI-MCI', 'CN-CN', 'CN-CN', 'MCI-MCI', 'MCI-AD',
       'AD-AD', 'CN-CN', 'AD-AD', 'MCI-MCI', 'CN-CN', 'CN-CN', 'CN-CN',
       'CN-CN', 'CN-CN', 'MCI-AD', 'CN-CN', 'MCI-AD', 'MCI-AD', 'MCI-AD',
       'MCI-AD', 'MCI-MCI', 'CN-CN', 'MCI-MCI', 'MCI-MCI', 'CN-CN',
       'AD-AD', 'CN-CN', 'CN-CN', 'CN-CN', 'CN-MCI', 'MCI-MCI', 'CN-CN',
       'MCI-MCI', 'MCI-MCI', 'MCI-MCI', 'AD-AD', 'MCI-MCI', 'CN-CN',
       'CN-CN', 'MCI-MCI', 'CN-CN', 'AD-AD', 'CN-CN', 'CN-CN', 'CN-MCI',
       'MCI-MCI', 'MCI-MCI', 'AD-AD', 'AD-AD', 'MCI-AD', 'MCI-AD',
       'MCI-MCI', 'MCI-MCI', 'CN-CN', 'MCI-MCI', 'CN-CN', 'MCI-MCI',
       'MCI-AD', 'MCI-AD', 'CN-CN', 'MCI-MCI', 'CN-CN', 'MCI-AD', 'CN-CN',
       'CN-CN', 'CN-CN', 'CN-CN', 'MCI-AD', 'CN-CN', 'MCI-AD', 'AD-AD',
       'CN-CN', 'MCI-AD', 'AD-AD', 'MCI-AD', 'MCI-AD', 'MCI-MCI', 'CN-CN',
       'MCI-AD', 'AD-AD', 'MCI-AD', 'AD-AD', 'CN-CN', 'AD-AD', 'MCI-MCI',
       'AD-AD', 'MCI-AD', 'MCI-MCI', 'CN-CN', 'AD-AD', 'MCI-MCI',
      

In [11]:
data = pd.DataFrame({
    'RowId': np.arange(1, len(predictions) + 1), 
    'Result': predictions
})

#data.to_csv('decision_tree_predicts_maxdepth6.csv', index=False)